In [1]:
import pulp
import folium
from pandas import DataFrame,read_csv
from numpy import hstack,newaxis,arange
from sklearn.neighbors import DistanceMetric
from geopandas import GeoDataFrame,points_from_xy
import warnings
warnings.filterwarnings('ignore')

# pack_id = 3496119
data = read_csv(r'C:\Users\m.aghili\Desktop\uni_project\LP\a.csv',delimiter='\t')
data2 = {
        'node_type':['dc','point','point','point','point','point','point','point','point','point','point','point','point','point','point','point','point','point','point'],
        'lat':[34.8591			,	34.8316046939023,	34.8228978181196,	34.8080493661635,	34.7994334608297,	34.7828666698494,	34.7906644574943,	34.7819627101561,	34.7675739146376,	34.7671672220595,	34.7742242261959,	34.7829669516897,	34.7853507335572,	34.7953593675638,	34.8069205283399,	34.8196409556211,	34.8071619417396,	34.8173314710403,	34.8367033586498], 
        'lon':[48.533,48.539198448039,48.5512363918378,48.5651911858679,48.5456425703998,48.5414574378181,48.5230141442578,48.5231491280914,48.5269689774552,48.5101138310113,48.4986695275898,48.5098959677305,48.4988854315506,48.4956127251111,48.4910542253773,48.5023152319877,48.5218864337468,48.5162038973198,48.5101496374482]
    }
data3 = {
        'node_type':['dc','point','point','point'],
        'lat':[34.8591,34.8069205283399,34.7864722047018,34.7819627101561], 
        'lon':[48.533,48.4910542253773,48.4866556882875,48.5231491280914]
    }
df = DataFrame(data3) 
df = GeoDataFrame(df, geometry=points_from_xy(df['lon'],df['lat']))
df.crs = "EPSG:4326"

def create_distance_matrix(lat, lon):
    haversine = DistanceMetric.get_metric('haversine')
    latlon = hstack((lat[:, newaxis], lon[:, newaxis]))
    dists = haversine.pairwise(latlon)
    return (6371 * dists).astype('int')

In [2]:
df

,node_type,lat,lon,geometry
0,dc,34.859100,48.533000,POINT (48.53300 34.85910)
1,point,34.806921,48.491054,POINT (48.49105 34.80692)
2,point,34.786472,48.486656,POINT (48.48666 34.78647)
3,point,34.781963,48.523149,POINT (48.52315 34.78196)


In [2]:
n_point = df.shape[0]
        
distances = create_distance_matrix(df.geometry.y,df.geometry.x)

problem = pulp.LpProblem('CVRP', pulp.LpMinimize)
#solver = pulp.CPLEX_PY()
x = pulp.LpVariable.dicts('x', ((i, j) for i in range(n_point) for j in range(n_point)), lowBound=0, upBound=1, cat='Binary') # edge
u = pulp.LpVariable.dicts('u', (i for i in range(n_point)), lowBound=1, upBound=n_point, cat='Integer') # Nodes
problem += pulp.lpSum(distances[i][j] * x[i, j] for i in range(n_point) for j in range(n_point)) # edge Cost
for i in range(n_point):
    problem += x[i, i] == 0 # there is no ring
for i in range(n_point):
    problem += pulp.lpSum(x[i, j] for j in range(n_point)) == 1 # only one edge comes out from each node
    problem += pulp.lpSum(x[j, i] for j in range(n_point)) == 1 # only one edge enters each node
for i in range(n_point):
    for j in range(n_point):
        if i != j and (i != 0 and j != 0):
            problem += u[i] - u[j] <= n_point * (1 - x[i, j]) - 1
status = problem.solve(pulp.PULP_CBC_CMD(maxSeconds=60))

In [3]:
problem

CVRP:
MINIMIZE
420*x_(0,_1) + 543*x_(0,_2) + 495*x_(0,_3) + 420*x_(1,_0) + 133*x_(1,_2) + 254*x_(1,_3) + 543*x_(2,_0) + 133*x_(2,_1) + 228*x_(2,_3) + 495*x_(3,_0) + 254*x_(3,_1) + 228*x_(3,_2) + 0
SUBJECT TO
_C1: x_(0,_0) = 0

_C2: x_(1,_1) = 0

_C3: x_(2,_2) = 0

_C4: x_(3,_3) = 0

_C5: x_(0,_0) + x_(0,_1) + x_(0,_2) + x_(0,_3) = 1

_C6: x_(0,_0) + x_(1,_0) + x_(2,_0) + x_(3,_0) = 1

_C7: x_(1,_0) + x_(1,_1) + x_(1,_2) + x_(1,_3) = 1

_C8: x_(0,_1) + x_(1,_1) + x_(2,_1) + x_(3,_1) = 1

_C9: x_(2,_0) + x_(2,_1) + x_(2,_2) + x_(2,_3) = 1

_C10: x_(0,_2) + x_(1,_2) + x_(2,_2) + x_(3,_2) = 1

_C11: x_(3,_0) + x_(3,_1) + x_(3,_2) + x_(3,_3) = 1

_C12: x_(0,_3) + x_(1,_3) + x_(2,_3) + x_(3,_3) = 1

_C13: u_1 - u_2 + 4 x_(1,_2) <= 3

_C14: u_1 - u_3 + 4 x_(1,_3) <= 3

_C15: - u_1 + u_2 + 4 x_(2,_1) <= 3

_C16: u_2 - u_3 + 4 x_(2,_3) <= 3

_C17: - u_1 + u_3 + 4 x_(3,_1) <= 3

_C18: - u_2 + u_3 + 4 x_(3,_2) <= 3

VARIABLES
1 <= u_1 <= 4 Integer
1 <= u_2 <= 4 Integer
1 <= u_3 <= 4 Integer
0 <= 

In [5]:
us_routes_df = DataFrame([[i, j] for i in range(n_point) for j in range(n_point) if pulp.value(x[i, j]) == 1])
us_routes_df

,0,1
0,0,1
1,1,2
2,2,3
3,3,0


In [6]:
routes = [0]
for i in range(len(us_routes_df)):
    routes.append(int(us_routes_df[us_routes_df[0]==routes[-1]][1]))
routes_df = DataFrame(routes,columns=['node_index_id'])
routes_df['priority'] = arange(routes_df.shape[0])+1
routes_df.set_index('node_index_id', inplace = True)
routes_df = GeoDataFrame(df.join(routes_df,how='left'))
routes_df.crs = "EPSG:4326"
routes_df.sort_values(by='priority', inplace=True)
routes_df['lat'] = routes_df['geometry'].y
routes_df['lon'] = routes_df['geometry'].x
routes_df.reset_index(drop=True, inplace=True)
routes_df

,node_type,lat,lon,geometry,priority
0,dc,34.859100,48.533000,POINT (48.53300 34.85910),1
1,point,34.806921,48.491054,POINT (48.49105 34.80692),2
2,point,34.786472,48.486656,POINT (48.48666 34.78647),3
3,point,34.781963,48.523149,POINT (48.52315 34.78196),4
4,dc,34.859100,48.533000,POINT (48.53300 34.85910),5


In [20]:
this_map = folium.Map(prefer_canvas=True)
folium.GeoJson(data=routes_df[:-1],name='node',tooltip=folium.GeoJsonTooltip(fields= ["priority"],aliases=["priority"],labels=True)).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map.add_child(folium.map.LayerControl())
this_map.save(r'C:\Users\m.aghili\Desktop\uni_project\LP\lp_core.html')